# Algo Trade Machine Learning End to End

This is an experiment to see if Machine Learning might be helpful to algorithm trading. While other team members working intensely on ML models, this notebook probes an end-to-end practical process yeilding actual trading actions, into which any premiumly trained ML model can be plug and play.

There are some special challenges in algo trade compared to other ML subjects. For example, we could only decide tentative labels which are far from actual answer, which means even if the ML model gets very high score, it still does not guarantee a good outcome. We'd need to go with some different kind of measurement. Another example is we are actually trying to predict the future, when no data is yet available. Instead of splitting train and test sets, we should only train then predict once at a time, like in the reality tomorrow comes one at a time. This requires some experimantal design targeting practicality as much as possible. 

## Experimental design

* One main ticker plus other supplemental tickers
* Gather historical data from Alpaca for 3 years up to today
* Start with 10K, process one trade action per day for "15 days" (action window)
* Predict one day at a time using previous day's data, having trained using "number of days backwards" (train window)
* While testing the trade actions use the day's market average price of open, close, high and low; If the instruction is to buy, we spend "the ticker's portion", i.e., the 1/"total number of tickers" of current available cash; If to sell, we sell all shares
* With multiple tickers action predicted, when the main ticker is not actionable, such as sell but no share, resort to the second ticker, then third, then forth, etc, to maximize the actions
* Probe the 15 days leading to today
* Move backwords one day at a time starting today, using the same sizes of train window and action window, run the training and prediction for 500 times
* Plot the distribution of the 500 experiments

## Define tickers

So far we use 4 random ones AAPL, AMZN, MSFT and GOOG. It would be easy to search and find other better tickers once we have the process built, and measurement decided

## Analysis for the algo

So far we use the simplest logic on volume weighted average price (vwap) changes for one day (daily return): gone down, buy; gone up, sell. Although we have the analysis step built in - looking for correlations amongst daily return, once and twice lagged daily return, there is not enough time for this project to try and find better curves and better logic. 
![curve for clue](Image/algo_trade_curve_for_clue.png)
 
## Support Vector Machine in 3 different ways

### Regression train for 90 days over once and twice lagged daily return (A)
![action a](Image/algo_trade_action_a.png)

![distribution a](Image/algo_trade_distribution_a.png)

### Classification train for 90 days over all data columns scaled with StandardScaler (B)
![action b](Image/algo_trade_action_b.png)

![distribution b](Image/algo_trade_distribution_b.png)

### Regression train for 45 days over all data columns percentage change (C)
![action c](Image/algo_trade_action_c.png)

![distribution c](Image/algo_trade_distribution_c.png)

## Random trade actions 
![action random](Image/algo_trade_action_random.png)

![distribution random](Image/algo_trade_distribution_random.png)

## Conclusion

For now it does not appear that any sure conclusion can be drawn. The method A shows the most chances to make some profit while slightly lower chance to lose; Method B looks like the random distrubution, slightly to the profit side; Method C shows more chance to make more profit, though all are low. 

We will go next steps including:
* Incoporate our primal ML models out from teammate's study
* Find better curves of ticker
* Refine algos to incoporate more parameters

## Major steps

* Train - where to plug in the ML model
```
def ML_pred_a( cycle ):
    """ one cycle of n_dates_for_training + 15 days, using SVR directly on vwap pct_change """
   
    # use lagged returns to train SVM
    data = cycle.loc[:, ['lagged_daily_return', 'twice_lagged']]
    label = cycle['signal']
    instrs = pd.DataFrame()
    model = svm.SVR()
    
    # rolling train for 30 days then predict 1 day, 15 cycles for 15 instructions
    for instri in range(0,15):
        iloc_to = instri + 90
        X = data.iloc[:iloc_to,:]
        y = label.iloc[:iloc_to]
        model.fit(X, y)
        signal = model.predict(data.iloc[iloc_to:iloc_to+1,:])
        
        if signal > 0:
            instr = 1
        else:
            instr = -1
        one_row = pd.DataFrame([[cycle.index[iloc_to].date(), instr]])
        instrs = pd.concat([instrs,one_row], join='outer')
        
    return instrs
```

* Predict - multi-ticker actions

![predicted actions](Image/algo_trade_predicted_action.png)

* Form instructions - one action per day

![actionable](Image/algo_trade_actionable.png)

* Action
* Plot distribution


In [566]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import alpaca_trade_api as tradeapi
from trade import trade_action
from datetime import datetime, timedelta
from sklearn import svm

In [567]:
def market_data( start, end, tickers ):
    load_dotenv()
    alpaca_api_key = os.getenv('ALPACA_API_KEY')
    alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

    # Create the Alpaca API object
    alpaca_api = tradeapi.REST(
       alpaca_api_key,
       alpaca_secret_key,
       api_version = 'v2'
    )
    
    # Set timeframe to "1Day" for Alpaca API
    timeframe = "1Day"
    
    start_date = pd.Timestamp(start,tz='America/New_York')
    end_date = pd.Timestamp(end,tz='America/New_York')
           
    # Get number_of_years' worth of historical data for tickers
    data_df = alpaca_api.get_bars(
        tickers,
        timeframe,
        start = start_date.isoformat(),
        end = end_date.isoformat()
    ).df
    if len(data_df) == 0:
        return []
    return data_df

#df_market = market_data( '2023-08-01', '2023-08-21', tickers=['TSLA'] )

In [484]:
def derive_instr( instrs ):
    buffer = [0]*4
    instr_df = pd.DataFrame()
    for index, instr in instrs.iterrows():
        #print(index, buffer, instr)
        one_row = pd.DataFrame()
        for idx in range(4):
            if buffer[idx]==1 and instr[idx]==-1:
                #print('In sell:', index, idx, -1)
                one_row = pd.DataFrame([[index, instr.index[idx], -1]])
                buffer[idx]=-1
                break
            elif buffer[idx]!=1 and instr[idx]==1:
                #print('In buy:',index, idx, 1)
                one_row = pd.DataFrame([[index, instr.index[idx], 1]])
                buffer[idx]=1
                break

        if len(one_row) == 0:
            one_row = pd.DataFrame([[index, instr.index[0], 0]])
        instr_df = pd.concat([instr_df, one_row])
    
    return instr_df

# my main

## 1. get market data

In [568]:
today = datetime.today().date()
tickers = ['AAPL', 'AMZN', 'MSFT', 'GOOG']
date_from = today - timedelta(days=365*3)
data_df = market_data(date_from, today, tickers)

dfs = {}
for ticker in tickers:
    df = data_df[data_df['symbol']==ticker]
    dfs[ticker] = df
                

## 2. prep data for algo

In [569]:
# signals is a df globally
# construct singals data:

signals_dfs = {}

for key in dfs:
    signals = dfs[key].loc[:,['vwap']]
    signals['daily_return'] = signals.pct_change()
    signals['lagged_daily_return'] = signals['daily_return'].shift(1)
    signals['twice_lagged'] = signals['daily_return'].shift(2)
    signals.dropna(inplace=True)
    signals_dfs[key]=signals
        
# visualize data prep for anz later
def visualize( ticker ):
    return signals_dfs[ticker][['daily_return','lagged_daily_return','twice_lagged']].hvplot(
                      title=ticker,
                      frame_width=700)
    
visualize('AAPL')

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

## 3. derive signal based on the observation of data

In [570]:
# derive signal - the singals df is global
for key in signals_dfs:
    # signals_dfs[key]['signal'] = np.where(
    #     (signals_dfs[key]['lagged_daily_return']>0) &
    #     (signals_dfs[key]['twice_lagged']>0), 
    #         -1, 
    #         np.where(
    #             (signals['lagged_daily_return']<0) &
    #             (signals['twice_lagged']<0), 1, 0 
    #                 )
    # )
    signals_dfs[key]['signal'] = np.where(
        signals_dfs[key]['lagged_daily_return']<0, 1, -1)


## 4. SVR on pct_change of vwap

### 4.1 the SVR ML

In [571]:
def ML_pred_a( cycle ):
    """ one cycle of n_dates_for_training + 15 days, using SVR directly on vwap pct_change """
   
    # use lagged returns to train SVM
    data = cycle.loc[:, ['lagged_daily_return', 'twice_lagged']]
    label = cycle['signal']
    instrs = pd.DataFrame()
    model = svm.SVR()
    
    # rolling train for 30 days then predict 1 day, 15 cycles for 15 instructions
    for instri in range(0,15):
        iloc_to = instri + 90
        X = data.iloc[:iloc_to,:]
        y = label.iloc[:iloc_to]
        model.fit(X, y)
        signal = model.predict(data.iloc[iloc_to:iloc_to+1,:])
        
        if signal > 0:
            instr = 1
        else:
            instr = -1
        one_row = pd.DataFrame([[cycle.index[iloc_to].date(), instr]])
        instrs = pd.concat([instrs,one_row], join='outer')
        
    return instrs

### 4.2 One cycle of 105 days (90+15)

In [572]:
instrs = pd.DataFrame()
for key in signals_dfs:
    cycle = signals_dfs[key].iloc[-105:,:]
    cycle_signals = ML_pred_a(cycle).set_index(0)
    cycle_signals.columns = [key]
    #instrs = pd.concat([instrs, one_cycle(cycle)])
    instrs = pd.concat([instrs, cycle_signals],axis=1)

instrs

,AAPL,AMZN,MSFT,GOOG
0,,,,
2023-08-14,1,1,1,1
2023-08-15,-1,-1,-1,-1
2023-08-16,1,1,-1,1
2023-08-17,1,1,1,1
2023-08-18,1,1,1,-1
2023-08-21,1,1,1,1
2023-08-22,-1,-1,-1,-1
2023-08-23,-1,-1,-1,-1
2023-08-24,-1,-1,-1,-1


#### 4.2.1 15-day trade actions

In [573]:
instr_df = derive_instr(instrs)
instr_df

,0,1,2
0,2023-08-14,AAPL,1
0,2023-08-15,AAPL,-1
0,2023-08-16,AAPL,1
0,2023-08-17,AMZN,1
0,2023-08-18,MSFT,1
0,2023-08-21,GOOG,1
0,2023-08-22,AAPL,-1
0,2023-08-23,AMZN,-1
0,2023-08-24,MSFT,-1
0,2023-08-25,AAPL,1


#### 4.2.2 show result

In [574]:
trade_action(instr_df)

On 2023-08-14 trade AAPL
{'action': 'buy', 'price': 178.61, 'bal': 7678.07, 'share': 13, 'status': 0, 'msg': 'success'}
On 2023-08-15 trade AAPL
{'action': 'sell', 'price': 178.21, 'bal': 9994.8, 'share': 13, 'status': 0, 'msg': 'success'}
On 2023-08-16 trade AAPL
{'action': 'buy', 'price': 177.18, 'bal': 7514.28, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-17 trade AMZN
{'action': 'buy', 'price': 134.76, 'bal': 5088.6, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-18 trade MSFT
{'action': 'buy', 'price': 315.23, 'bal': 2881.99, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-21 trade GOOG
{'action': 'buy', 'price': 128.3, 'bal': 444.29, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-22 trade AAPL
{'action': 'sell', 'price': 177.06, 'bal': 2923.13, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-23 trade AMZN
{'action': 'sell', 'price': 134.8, 'bal': 5349.53, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-24 trade MSFT
{'action': 'sell',

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

### 4.3 rolling run 500 times

In [ ]:
def one_run_a(str_loc):
    instrs = pd.DataFrame()
    for key in signals_dfs:
        cycle = signals_dfs[key].iloc[str_loc:,:]
        signals_dfs[key].iloc[str_loc:, :]
        cycle_signals = ML_pred_a(cycle).set_index(0)
        cycle_signals.columns = [key]
        instrs = pd.concat([instrs, cycle_signals],axis=1)
        
    instr_df = derive_instr(instrs)
    return trade_action(instr_df, verbose=0)

final_worths = []
for n in range(500):
    final_worths.append( one_run_a(-105-n) )


In [596]:
# plot distribution
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[('final_worth','@final_worth{0.0f}')])
pd.DataFrame(final_worths, columns=['final_worth']).hvplot(kind='hist', xformatter="%.0f", tools=[hover])

:Histogram   [final_worth]   (final_worth_count)

## 5. SVC on scaled all columns

In [597]:
# shifted is a df globally. shift(3) is required to match signal (daily return, lagged, twice_lagged)
# shifted data is for 
shifted_dfs = {}
for key in ticker_data:
    shifted_dfs[key] = dfs[key].drop('symbol',axis=1).shift(3).dropna()

### 5.1 the SVC ML

In [598]:
from sklearn.preprocessing import StandardScaler

def ML_pred_b( data, label ):
    """ one cycle of n_dates_for_training + 15 days, using SVR on all cols """
   
    # scale for X
    scaler = StandardScaler()
    scaler.fit(data)
    all_data = pd.DataFrame(scaler.transform(data))
    all_label = label
    
    instrs = pd.DataFrame()
    model = svm.SVC(kernel='rbf')
    
    # rolling train for 30 days then predict 1 day, 15 cycles for 15 instructions
    for instri in range(0,15):
        iloc_to = instri + 90
        X = all_data.iloc[:iloc_to,:]
        y = all_label.iloc[:iloc_to]
        model.fit(X, y)
        signal = model.predict(all_data.iloc[iloc_to:iloc_to+1,:])
        instr = signal[0]
        one_row = pd.DataFrame([[data.index[iloc_to].date(), instr]])
        instrs = pd.concat([instrs,one_row], join='outer')
        
    return instrs

### 5.2 one cycle of 105 days

In [599]:
instrs = pd.DataFrame()
for key in shifted_dfs:
    cycle = shifted_dfs[key].iloc[-105:,:]
    data = cycle
    label = signals_dfs[key]['signal']
    cycle_signals = ML_pred_b(data, label).set_index(0)
    cycle_signals.columns = [key]
    instrs = pd.concat([instrs, cycle_signals],axis=1)

instr_df = derive_instr(instrs)
trade_action(instr_df)

On 2023-08-14 trade AAPL
{'action': 'buy', 'price': 178.61, 'bal': 7678.07, 'share': 13, 'status': 0, 'msg': 'success'}
On 2023-08-15 trade AMZN
{'action': 'buy', 'price': 139.06, 'bal': 5314.05, 'share': 17, 'status': 0, 'msg': 'success'}
On 2023-08-16 trade MSFT
{'action': 'buy', 'price': 321.36, 'bal': 3064.53, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-17 trade GOOG
{'action': 'buy', 'price': 130.81, 'bal': 579.14, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-18 trade AAPL
{'action': 'none', 'price': 173.46, 'bal': 579.14, 'share': 13, 'status': -1, 'msg': 'invalid action (1/-1 only)'}
On 2023-08-21 trade MSFT
{'action': 'sell', 'price': 319.9, 'bal': 2818.44, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-22 trade AAPL
{'action': 'none', 'price': 177.06, 'bal': 2818.44, 'share': 13, 'status': -1, 'msg': 'invalid action (1/-1 only)'}
On 2023-08-23 trade AAPL
{'action': 'none', 'price': 179.88, 'bal': 2818.44, 'share': 13, 'status': -1, 'msg': 'invalid ac

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

### 5.3 rolling run 500 times

In [600]:
def one_run_b(str_loc):
    instrs = pd.DataFrame()
    for key in shifted_dfs:
        cycle = shifted_dfs[key].iloc[str_loc:,:]
        cycle_signals = ML_pred_b(cycle, signals_dfs[key]['signal']).set_index(0)
        cycle_signals.columns = [key]
        instrs = pd.concat([instrs, cycle_signals],axis=1)
        
    instr_df = derive_instr(instrs)
    return trade_action(instr_df, verbose=0)

final_worths = []
for n in range(500):
    final_worths.append( one_run_b(-105-n) )


In [601]:
# plot distribution
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[('final_worth','@final_worth{0.0f}')])
pd.DataFrame(final_worths, columns=['final_worth']).hvplot(kind='hist', xformatter="%.0f", tools=[hover])

:Histogram   [final_worth]   (final_worth_count)

## 6. svr on pct_change of all columns

In [603]:
# shifted is a df globally. first we cal pct_change, then shift(2) is required to match signal (daily return, lagged, twice_lagged)
shifted_dfs = {}
for key in ticker_data:
    shifted_dfs[key] = dfs[key].drop('symbol',axis=1).pct_change().shift(2)
    shifted_dfs[key].dropna(inplace=True)


### 6.1 train and predict

In [604]:
def ML_pred_c( data, label ):
    """ one cycle of n_dates_for_training + 15 days, using SVR on pct_change of all cols """
   
    # use lagged returns to train SVM
    instrs = pd.DataFrame()
    model = svm.SVR()
    
    # rolling train for 30 days then predict 1 day, 15 cycles for 15 instructions
    for instri in range(0,15):
        iloc_to = instri + 30
        X = data.iloc[:iloc_to,:]
        y = label.iloc[:iloc_to]
        model.fit(X, y)
        signal = model.predict(data.iloc[iloc_to:iloc_to+1,:])
        
        if signal > 0:
            instr = 1
        else:
            instr = -1
        #one_row = pd.DataFrame([[signals.index[iloc_to].date(), cycle['symbol'][0], instr]])
        one_row = pd.DataFrame([[cycle.index[iloc_to].date(), instr]])
        instrs = pd.concat([instrs,one_row], join='outer')
        
    return instrs

### 6.2 one cycle of 45 days

In [605]:
instrs = pd.DataFrame()
for key in shifted_dfs:
    cycle = shifted_dfs[key].iloc[-45:,:]
    cycle_signals = ML_pred_c(cycle, signals_dfs[key]['signal']).set_index(0)
    cycle_signals.columns = [key]
    instrs = pd.concat([instrs, cycle_signals],axis=1)

# create workable instructions
instr_df = derive_instr(instrs)

# show result
trade_action(instr_df)

On 2023-08-14 trade AAPL
{'action': 'buy', 'price': 178.61, 'bal': 7678.07, 'share': 13, 'status': 0, 'msg': 'success'}
On 2023-08-15 trade AMZN
{'action': 'buy', 'price': 139.06, 'bal': 5314.05, 'share': 17, 'status': 0, 'msg': 'success'}
On 2023-08-16 trade GOOG
{'action': 'buy', 'price': 129.44, 'bal': 2854.69, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-17 trade MSFT
{'action': 'buy', 'price': 318.88, 'bal': 622.53, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-18 trade AAPL
{'action': 'sell', 'price': 173.46, 'bal': 2877.51, 'share': 13, 'status': 0, 'msg': 'success'}
On 2023-08-21 trade GOOG
{'action': 'sell', 'price': 128.3, 'bal': 5315.21, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-22 trade AAPL
{'action': 'buy', 'price': 177.06, 'bal': 2836.37, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-23 trade GOOG
{'action': 'buy', 'price': 132.16, 'bal': 457.49, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-24 trade MSFT
{'action': 'sell'

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

### 6.3 rolling run 500 times

In [606]:
def one_run_c(str_loc):
    instrs = pd.DataFrame()
    for key in shifted_dfs:
        cycle = shifted_dfs[key].iloc[str_loc:,:]
        cycle_signals = ML_pred_c(cycle, signals_dfs[key]['signal']).set_index(0)
        cycle_signals.columns = [key]
        instrs = pd.concat([instrs, cycle_signals],axis=1)
        
    instr_df = derive_instr(instrs)
    return trade_action(instr_df, verbose=0)

final_worths = []
for n in range(500):
    final_worths.append( one_run_c(-45-n) )


In [607]:
# plot distribution
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[('final_worth','@final_worth{0.0f}')])
pd.DataFrame(final_worths, columns=['final_worth']).hvplot(kind='hist', xformatter="%.0f", tools=[hover])

:Histogram   [final_worth]   (final_worth_count)

## 7. random trade action

### 7.1 randomly generate trade actions

In [608]:
instrs = np.random.randn(50, 1)

def sample_df():
    instri = 0

    # using alpaca data for the dates to avoid non-trading days
    df_market = dfs[tickers[0]].iloc[-15:,:]
    
    df = pd.DataFrame()
    for index, data in df_market.iterrows():
        if instrs[instri] > 0:
            instr = 1
        else:
            instr = -1
        instri += 1
        one_row = pd.DataFrame([[index.date(), tickers[np.random.randint(4)], instr]])
        df = pd.concat([df,one_row], join='outer')
    return df

random_actions = sample_df()
random_actions

,0,1,2
0,2023-08-14,GOOG,-1
0,2023-08-15,AAPL,1
0,2023-08-16,AMZN,1
0,2023-08-17,MSFT,-1
0,2023-08-18,MSFT,-1
0,2023-08-21,AAPL,-1
0,2023-08-22,AAPL,-1
0,2023-08-23,GOOG,1
0,2023-08-24,GOOG,1
0,2023-08-25,AMZN,-1


### 7.2 show result

In [609]:
trade_action(random_actions)

On 2023-08-14 trade GOOG
{'action': 'none', 'price': 130.8, 'bal': 10000.0, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-15 trade AAPL
{'action': 'buy', 'price': 178.21, 'bal': 7505.06, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-16 trade AMZN
{'action': 'buy', 'price': 136.14, 'bal': 5054.54, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-17 trade MSFT
{'action': 'none', 'price': 318.88, 'bal': 5054.54, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-18 trade MSFT
{'action': 'none', 'price': 315.23, 'bal': 5054.54, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-21 trade AAPL
{'action': 'sell', 'price': 175.19, 'bal': 7507.2, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-22 trade AAPL
{'action': 'none', 'price': 177.06, 'bal': 7507.2, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-23 trade GOOG
{'action': 'buy', 'price': 132.16, 'bal': 5128.32, 'share': 18, 'status': 0, 'msg': 'success'}
On

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

### 7.3 run 500 times random actions see distribution

In [610]:
final_worths = []
for n in range(500):
    final_worths.append(trade_action( sample_df(), verbose=0 ))


In [611]:
# plot distribution
from bokeh.models import HoverTool
hover = HoverTool(tooltips=[('final_worth','@final_worth{0.0f}')])
pd.DataFrame(final_worths, columns=['final_worth']).hvplot(kind='hist', xformatter="%.0f", tools=[hover])

:Histogram   [final_worth]   (final_worth_count)